# Tutorial 4: Create Athena tables from 

To enable valuable data searches, we must align the sample-level metadata appended to the raw data packages to the pipeline outputs. In this demo, the primary data generated by the pipeline is expression tables. With Athena, its possible to integrate sample metadata & pipeline output tables together to empower quick queries and slicing and dicing of large datasets.

Now that we have raw data pacakaged with sample-level metadata, and Nextflow pipeline outputs packaged with `nf-quilt`, the next step is use this data to create tables and views in Athena to enable snappy data queries. 

For more more information on querying Quilt package metadata in Athena, please refer to the Quilt documentation at <https://docs.quiltdata.com/advanced/athena>.



In [ ]:
import boto3 as boto3

# Connect to Athena

In [ ]:
# user to fill in AWS credentials (or auth in preferred method)
aws_access_key_id = "USER_ACCESS KEY"
aws_secret_access_key = "USER SECRET ACCESS KEY"
region_name = "us-east-1"
bucket_name = "s3://quilt-example-bucket/ccle"

In [1]:
# Connect to Athena
conn = connect(aws_access_key_id=aws_access_key_id,
               aws_secret_access_key=aws_secret_access_key,
               region_name=region_name,
               s3_staging_dir= bucket_name)

# Generate symlink datalake across Nextflow pipeline runs

Typically Athena and AWS Glue prefer data to be organized in a specific structure, usually 1 file per directory, where the directory levels represent a partition key. `nf-core` Nextflow pipelines do not output data in the traditionally Athena compatible structure, with many files per directory and directories often representing the names of different tools/steps in the pipeline. 

As an alternative to re-structuring or copying pipeline outputs to be compatible with Athena organization, in this demo we use symbolic links (symlinks) to create a datalake of Nextflow pipeline outputs partitioned by `run_id` and `sample_id`. We then use this data lake to create a view representing a table of each samples gene expression values from a `quant.sf` pipeline output across all samples across all runs. 

In [ ]:
s3 = boto3.resource('s3')
bucket_name = "quilt-example-bucket"
prefix = "ccle/datalake"

In [ ]:
# define the quant.sf files we want to use to populate the datalake
# a similar method can be used for other piepline outputs
parse_keys = """
WITH extracted_data AS (
    SELECT
        timestamp,
        logical_key,
        regexp_extract(logical_key, '^[^/]+/([^/]+)/.*', 1) AS sample_id,
        physical_key,
        regexp_extract(physical_key, '^(.*?)\?versionId=', 1) AS object_path
    FROM
        "userathenadatabase-2htmlbiqyvry"."quilt-example-bucket_objects-view"
    WHERE
        logical_key LIKE '%quant.sf'
)
SELECT
    logical_key,
    sample_id,
    physical_key,
    object_path
FROM
    extracted_data
WHERE
    sample_id LIKE 'SRR%';
    AND timestamp = 'latest';
"""

cursor = conn.cursor()
cursor.execute(parse_keys)

In [ ]:
# generate a symlinks file for each file of interest
for row in cursor:
        sample_id = row[1]
        run_id = row[3].split("/")[4]
        object_path = row[3]
        s3.Bucket(bucket_name).put_object(Key=f"{prefix}/run_id={run_id}/sample_id={sample_id}/symlink.txt", Body=object_path)

In [ ]:
# use the symlink data lake to create a new table containins samples x gene expression across all runs
parse_keys = """
CREATE EXTERNAL TABLE `ccle_nfcore_rnaseq_datalake`(
  `name` string COMMENT 'from deserializer', 
  `length` string COMMENT 'from deserializer', 
  `effectivelength` string COMMENT 'from deserializer', 
  `tpm` string COMMENT 'from deserializer', 
  `numreads` string COMMENT 'from deserializer')
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
WITH SERDEPROPERTIES ( 
  'separatorChar'='\t') 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://quilt-example-bucket/ccle/datalake/'
  tblproperties (
  'skip.header.line.count'='1'
)
"""
cursor = conn.cursor()
cursor.execute(parse_keys)

# Create table for sample-level metadata

Quilt package metadata is stored on s3 as a json file. We can use this metadata to create a table in Athena. Below is example SQL code to create this table view in Athena , which will output a sample x field table for all the sample-level metadata appended to raw data packages in Quilt. 


```sql
--- create expanded view of raw pkg metadata
CREATE OR REPLACE VIEW ccle_raw_data_pkg_metadata AS
SELECT
    *,
    JSON_EXTRACT_SCALAR(user_meta, '$.lineage') AS lineage,
    JSON_EXTRACT_SCALAR(user_meta, '$.createdate') AS createdate,
    JSON_EXTRACT_SCALAR(user_meta, '$.run')AS run,
    JSON_EXTRACT_SCALAR(user_meta, '$.tissue')AS tissue,
    JSON_EXTRACT_SCALAR(user_meta, '$.primarytumorsite') AS primarytumorsite,
    JSON_EXTRACT_SCALAR(user_meta, '$.releasedate')AS releasedate,
    JSON_EXTRACT_SCALAR(user_meta, '$.mutationrate')AS mutationrate,
    JSON_EXTRACT_SCALAR(user_meta, '$.cellline') AS cellline,
    JSON_EXTRACT_SCALAR(user_meta, '$.celllinenickname') AS celllinenickname,
    JSON_EXTRACT_SCALAR(user_meta, '$.oncotreecode')AS oncotreecode,
    JSON_EXTRACT_SCALAR(user_meta, '$.experiment') AS experiment,
    JSON_EXTRACT_SCALAR(user_meta, '$.histology') AS histology,
    JSON_EXTRACT_SCALAR(user_meta, '$.tmbnonsynonymous') AS tmbnonsynonymous,
    JSON_EXTRACT_SCALAR(user_meta, '$.avgspotlen') AS avgspotlen,
    JSON_EXTRACT_SCALAR(user_meta, '$.datastoreprovider') AS datastoreprovider,
    JSON_EXTRACT_SCALAR(user_meta, '$.srastudy') AS srastudy,
    JSON_EXTRACT_SCALAR(user_meta, '$.biosample') AS biosample,
    JSON_EXTRACT_SCALAR(user_meta, '$.pathologistannotation') AS pathologistannotation,
    JSON_EXTRACT_SCALAR(user_meta, '$.samplename') AS samplename,
    JSON_EXTRACT_SCALAR(user_meta, '$.doublingtimehrs') AS doublingtimehrs,
    JSON_EXTRACT_SCALAR(user_meta, '$.organism') AS organism,
    JSON_EXTRACT_SCALAR(user_meta, '$.genomedoublings') AS genomedoublings,
    JSON_EXTRACT_SCALAR(user_meta, '$.biosamplemodel') AS biosamplemodel,
    JSON_EXTRACT_SCALAR(user_meta, '$.sitesubtype1') AS sitesubtype1,
    JSON_EXTRACT_SCALAR(user_meta, '$.sitesubtype2') AS sitesubtype2,
    JSON_EXTRACT_SCALAR(user_meta, '$.datastorefiletype') AS datastorefiletype,
    JSON_EXTRACT_SCALAR(user_meta, '$.datastoreregion') AS datastoreregion,
    JSON_EXTRACT_SCALAR(user_meta, '$.fractiongenomealtered') AS fractiongenomealtered,
    JSON_EXTRACT_SCALAR(user_meta, '$.consent') AS consent,
    JSON_EXTRACT_SCALAR(user_meta, '$.centername') AS centername,
    JSON_EXTRACT_SCALAR(user_meta, '$.ploidy') AS ploidy,
    JSON_EXTRACT_SCALAR(user_meta, '$.freezingmedium')AS freezingmedium,
    JSON_EXTRACT_SCALAR(user_meta, '$.siteoffinding') AS siteoffinding,
    JSON_EXTRACT_SCALAR(user_meta, '$.growthmedium') AS growthmedium,
    JSON_EXTRACT_SCALAR(user_meta, '$.annotationsource') AS annotationsource,
    JSON_EXTRACT_SCALAR(user_meta, '$.ethnicity') AS ethnicity,
    JSON_EXTRACT_SCALAR(user_meta, '$.assemblyname') AS assemblyname,
    JSON_EXTRACT_SCALAR(user_meta, '$.biomaterialprovider') AS biomaterialprovider,
    JSON_EXTRACT_SCALAR(user_meta, '$.purity') AS purity,
    JSON_EXTRACT_SCALAR(user_meta, '$.instrument') AS instrument,
    JSON_EXTRACT_SCALAR(user_meta, '$.cancertypedetailed') AS cancertypedetailed,
    JSON_EXTRACT_SCALAR(user_meta, '$.platform') AS platform,
    JSON_EXTRACT_SCALAR(user_meta, '$.cancertype') AS cancertype,
    JSON_EXTRACT_SCALAR(user_meta, '$.lineagesubtype') AS lineagesubtype,
    JSON_EXTRACT_SCALAR(user_meta, '$.bases') AS bases,
    JSON_EXTRACT_SCALAR(user_meta, '$.flowcellid') AS flowcellid,
    JSON_EXTRACT_SCALAR(user_meta, '$.diseasestage') AS diseasestage,
    JSON_EXTRACT_SCALAR(user_meta, '$.isolate') AS isolate,
    JSON_EXTRACT_SCALAR(user_meta, '$.assaytype') AS assaytype,
    JSON_EXTRACT_SCALAR(user_meta, '$.histologysubtype1') AS histologysubtype1,
    JSON_EXTRACT_SCALAR(user_meta, '$.subtype') AS subtype,
    JSON_EXTRACT_SCALAR(user_meta, '$.createdatebatch') AS createdatebatch,
    JSON_EXTRACT_SCALAR(user_meta, '$.histologysubtype2') AS histologysubtype2,
    JSON_EXTRACT_SCALAR(user_meta, '$.studyid') AS studyid,
    JSON_EXTRACT_SCALAR(user_meta, '$.depmapid') AS depmapid,
    JSON_EXTRACT_SCALAR(user_meta, '$.librarylayout') AS librarylayout,
    JSON_EXTRACT_SCALAR(user_meta, '$.characteristics') AS characteristics,
    JSON_EXTRACT_SCALAR(user_meta, '$.disease') AS disease,
    JSON_EXTRACT_SCALAR(user_meta, '$.patientid') AS patientid,
    JSON_EXTRACT_SCALAR(user_meta, '$.sex') AS sex,
    JSON_EXTRACT_SCALAR(user_meta, '$.celllinesource') AS celllinesource,
    JSON_EXTRACT_SCALAR(user_meta, '$.librarysource') AS librarysource,
    JSON_EXTRACT_SCALAR(user_meta, '$.libraryselection') AS libraryselection,
    JSON_EXTRACT_SCALAR(user_meta, '$.bioproject') AS bioproject,
    JSON_EXTRACT_SCALAR(user_meta, '$.libraryname') AS libraryname,
    JSON_EXTRACT_SCALAR(user_meta, '$.sampletype') AS sampletype,
    JSON_EXTRACT_SCALAR(user_meta, '$.mutationcount') AS mutationcount,
    JSON_EXTRACT_SCALAR(user_meta, '$.age') AS age
FROM
    "userathenadatabase-2htmlbiqyvry"."quilt-example-bucket_packages-view"
WHERE 
    substr(pkg_name, -10, 3) = 'SRR'
    AND timestamp = 'latest';

--- preview new view
SELECT * FROM "userathenadatabase-2htmlbiqyvry"."ccle_raw_data_pkg_metadata" limit 10;

```

Now we can use sample ID as the join key to join these sample-level metadata directly to nf-core pipeline tables created from the symlink datalake.

# 